In [1]:
import pandas as pd

## Data Extraction

The MEPS data files can be found [here](https://meps.ahrq.gov/mepsweb/data_stats/download_data_files.jsp).  The full data consists of numerous tables with various levels of granularity, and you can read more about data under the Data Overview section.  For our purposes, we will be working with the "Household Full Year Consolidated Data File" datasets.  Let's download the latest version (2015) ASCII zip file and unzip.

#### What you will accomplish
In this notebook, you will learn to read, parse, and provide structure to a raw dataset.  The output will be csv datasets for the 2014 and 2015 MEPS data.

### Inspecting and Parsing Raw Data

First let's inspect the raw data. You'll notice that each row is a long text string where different positional values correspond to different data fields.  

Open the data file and print the first line.

In [2]:
# Print the first line of the raw data file
with open("/Users/amy/Downloads/h181.dat") as data2015:
    lines = data2015.readlines()
    
firstline = lines[0]
firstline

'600011016000110119A A A A A A 0101A A A A 0505050501010110505050501050510202020210110110110111110101011010820140220150220150820150820151220151220151111111111111111160606052525353531119621101331229050505059959959959950202020203-1-113-1-1-1-10202020202020101010102-1-101-1-01-01-01-01-01-01040404010103014601014601470146014702-101460202014602-1-1-1-1-1-1-1-1-1-1-1-10146010101035002-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1020202020102010201-101-102-102-102-101-101-101-1010101010202020201010202020202-102-102010202020101-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1.0-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-10401010101010101-1-10101-1-1-1-10101030107-101045.3011001010401040601040404040401040301030902-101010304010103030505030403040525.9358.470001000200000003000000010101010820150101020101010102010101010201010101020201020202020201010101010201020402-1-1-1-10101-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1030301090102010202020202020101010104010401-804

We will need to cross-reference this file with the corresponding Codebook to determine the location of fields we are interested in.  Using this codebook, can you identify the age of the first individual in the data (remember: the codebook indexing starts at 1 but python indexing starts at 0).

In [3]:
# Print the age of the first person in the data
age_s = 178
age_e = 180
int(firstline[age_s:age_e])

53

The different indexing between the codebook and our data can be confusing.  How can you sanity check whether you have properly parsed age?

In [4]:
# e.g. Does the distribution of ages across the entire dataset seem reasonable?


You will notice that all of the values in the data file are integers.  Since age is also an integer, we don't need to do additional parsing.  However, other fields, like sex, are categorical, and the codebook provides an additional mapping from integer to category label.  Print out the sex of the first observation in the dataset

In [5]:
# What is the sex of the first observation?
sexpos = 186
sexcode = int(firstline[sexpos])
if sexcode == 1:
    sex = 'male'
elif sexcode == 2:
    sex = 'female'
sex

'male'

As we pull out more features, it will be useful to store this extraction logic in a function.  Write a function that takes in a row of raw text data, and returns a dictionary with the fields we have extracted so far.  You will also want to extract a unique identifier for each individual. The DUPERSID field serves this purpose.

In [11]:
with open("/Users/amy/Downloads/h181.dat") as data2015:
    lines = data2015.readlines()
    

def parse_row_2015(row):
    """
    Write a function to extract and label data from raw 2015 MEPS
    data
    """
    d = {}
    # Add age
    # AGELAST = 179-180
    age_s = 178
    age_e = 180
    d['age'] = int(row[age_s:age_e])
    
    # Add sex
    # SEX = 187
    sexpos = 186
    sexcode = int(row[sexpos])
    if sexcode == 1:
        d['sex'] = 'male'
    elif sexcode == 2:
        d['sex'] = 'female'
    
    # Add ID
    # DUPERSID = 9-16
    id_s = 8
    id_e = 16
    d['user_id'] = int(row[id_s:id_e])
    
    # Add Race
    # RACEV1X = 188
    racepos = 187
    racecode = int(row[racepos])
    if racecode == 1:
        d['race'] = 'White'
    elif racecode == 2:
        d['race'] = 'Black'
    elif racecode == 3:
        d['race'] = 'American Indian/Alaska Native'
    elif racecode == 4:
        d['race'] = 'Asian/Native Hawaiian/Pacific Islander'
    elif racecode == 6:
        d['race'] = 'Multiple Races Reported'
    
    # Add Total Expenditures (self and third-party)
    # TOTEXP15 = 2563-2568
    totexp_s = 2562
    totexp_e = 2568
    d['tot_exp'] = int(row[totexp_s:totexp_e])
    
    # Add Total out-of-pocket payments
    # TOTSLF15 = 2569-2573
    totslf_s = 2568
    totslf_e = 2573
    d['tot_slf_exp'] = int(row[totslf_s:totslf_e])
    
    # Add Total paid by Medicaid
    # TOTMCD15 = 2580-2585
    totmcd_s = 2579
    totmcd_e = 2585
    d['tot_mcd_exp'] = int(row[totmcd_s:totmcd_e])
    
    # Add Total paid by Medicare
    # TOTMCR15 = 2574-2579
    totmcr_s = 2573
    totmcr_e = 2579
    d['tot_mcr_exp'] = int(row[totmcr_s:totmcr_e])
    
    # Add Total paid by Private insurance
    # TOTPRV15 = 2586-2591
    totprv_s = 2585
    totprv_e = 2591
    d['tot_prv_exp'] = int(row[totprv_s:totprv_e])
    
    # Insurance coverage status
    # Insurance code to status helper function
    def insur_code_to_desc(code):
        if code == 1:
            return 'Yes'
        elif code == 2:
            return 'No'
        
    # Private Insurance (did you ever have private insurance during 2015?)
    # PRVEV15 = 2167
    # Is this the right metric to use?? Look into making this a categorical variable
    prvev = 2166
    prvev_code = int(row[prvev])
    d['insur_priv'] = insur_code_to_desc(prvev_code)
    
    # Medicare in 2015
    # MCREV15 = 2169
    mcrev = 2168
    mcrev_code = int(row[mcrev])
    d['insur_mcr'] = insur_code_to_desc(mcrev_code)
    
    # Medicaid in 2015
    # MCDEV15 = 2170
    mcdev = 2169
    mcdev_code = int(row[mcdev])
    d['insur_mcd'] = insur_code_to_desc(mcdev_code)
        
    # Perceived Health Rating helper function
    def self_rate_to_desc(rating):
        if rating == -9:
            return 'Not Ascertained'
        elif rating == -8:
            return 'DK'
        elif rating == -7:
            return 'Refused'
        elif rating == -1:
            return 'Inapplicable'
        elif rating == 1:
            return 'Excellent'
        elif rating == 2:
            return 'Very Good'
        elif rating == 3:
            return 'Good'
        elif rating == 4:
            return 'Fair'
        elif rating == 5:
            return 'Poor'
    
    # Add Perceived Health
    # RTHLTH31 = 289-290
    rh31_s = 288
    rh31_e = 290
    rh31_rate = int(row[rh31_s:rh31_e])
    d['rh31'] = self_rate_to_desc(rh31_rate)
    # RTHLTH42 = 291-292
    rh42_s = 290
    rh42_e = 292
    rh42_rate = int(row[rh42_s:rh42_e])
    d['rh42'] = self_rate_to_desc(rh42_rate)
    # RTHLTH53 = 293-294
    rh53_s = 292
    rh53_e = 294
    rh53_rate = int(row[rh53_s:rh53_e])
    d['rh53'] = self_rate_to_desc(rh53_rate)
    
    # Add Perceived Mental Health
    # MNHLTH31 = 295-296
    mh31_s = 294
    mh31_e = 296
    mh31_rate = int(row[mh31_s:mh31_e])
    d['mh31'] = self_rate_to_desc(mh31_rate)
    # MNHLTH42 = 297-298
    mh42_s = 296
    mh42_e = 298
    mh42_rate = int(row[mh42_s:mh42_e])
    d['mh42'] = self_rate_to_desc(mh42_rate)
    # MNHLTH53 = 299-300
    mh53_s = 298
    mh53_e = 300
    mh53_rate = int(row[mh53_s:mh53_e])
    d['mh53'] = self_rate_to_desc(mh53_rate)
    
    # Create rating columns to describe change in self-assessed health
    # Rating change helper function
    def sah_rating_change(rate1, rate2):
        if (rate1 < 1) | (rate2 < 1):
            return "N/A"
        else:
            return rate2 - rate1
    
    # RH31 to RH42 Change
    d['rh31_42_chnge'] = sah_rating_change(rh31_rate, rh42_rate)
    # RH42 to RH53 Change
    d['rh42_53_chnge'] = sah_rating_change(rh42_rate, rh53_rate)
    # Total RH change
    d['rh_tot_chnge'] = sah_rating_change(rh31_rate, rh53_rate)
    # MH31 to MH42 change
    d['mh31_42_chnge'] = sah_rating_change(mh31_rate, mh42_rate)
    # MH42 to MH53 change
    d['mh42_53_chnge'] = sah_rating_change(mh42_rate, mh53_rate)
    # Total MH change
    d['mh_tot_chnge'] = sah_rating_change(mh31_rate, mh53_rate)
    
    # Diagnosis Rating helper function
    def diag_rating_to_desc(rating):
        if rating == -9:
            return "Not Ascertained"
        elif rating == -8:
            return "DK"
        elif rating == -7:
            return "Refused"
        elif rating == -1:
            return "Inapplicable"
        elif rating == 1:
            return "Yes"
        elif rating == 2:
            return "No"
    
    # Add Alzheimers/dementia Diagnosis - DID NOT HAVE
    
    # Add Arthritis Diagnosis
    # ARTHDX = 373-374
    arth_s = 372
    arth_e = 374
    d['dx_arth'] = diag_rating_to_desc(int(row[arth_s:arth_e]))
    
    # Add Atrial fibrilation Diagnosis - DID NOT HAVE
    
    # Add Asthma Diagnosis
    # ASTHDX = 379-380
    asth_s = 378
    asth_e = 380
    d['dx_asth'] = diag_rating_to_desc(int(row[asth_s:asth_e]))
    
    # Add Autism Diagnosis - not explicitly there, ask about CHCOUN42
    
    # Add Cancer Diagnosis
    # CANCERDX = 339-340
    canc_s = 338
    canc_e = 340
    d['dx_canc'] = diag_rating_to_desc(int(row[canc_s:canc_e]))
    
    # Add Chronic kidney disease Diagnosis - only w/r/t diabetes
    
    # Add Copd (Chronic Obstructive Pulmonary Disease) Diagnosis (Emphysema)
    # EMPHDX = 327-328
    copd_s = 326
    copd_e = 328
    d['dx_COPD'] = diag_rating_to_desc(int(row[copd_s:copd_e]))
    
    # Add Depression Diagnosis - did not have per se (have you felt depressed in last 4 weeks SAQ)
    
    # Add Diabetes Diagnosis
    # DIABDX = 365-366
    diab_s = 364
    diab_e = 366
    d['dx_diab'] = diag_rating_to_desc(int(row[diab_s:diab_e]))
    
    # Add heart failure Diagnosis - /heart attack

    # Add hepatitis Diagnosis - DID NOT HAVE

    # Add hiv/aids Diagnosis - DID NOT HAVE

    # Add Hyperlipidemia Diagnosis (high cholesterol)
    # CHOLDX = 335-336
    hyplip_s = 334
    hyplip_e = 336
    d['dx_hyplp'] = diag_rating_to_desc(int(row[hyplip_s:hyplip_e]))

    # Add Hypertension Diagnosis (high blood pressure)
    # HIBPDX = 301-302
    hypt_s = 300
    hypt_e = 302
    d['dx_hyptn'] = diag_rating_to_desc(int(row[hypt_s:hypt_e]))

    # Add ischemic heart disease Diagnosis (coronary heart disease?)
    # CHDDX = 307-308
    ihd_s = 306
    ihd_e = 308
    d['dx_IHD'] = diag_rating_to_desc(int(row[ihd_s:ihd_e]))

    # Add osteoporosis Diagnosis - DID NOT HAVE

    # Add schizophrenia Diagnosis - DID NOT HAVE

    # Add stroke Diagnosis
    # STRKDX = 323-324
    strk_s = 322
    strk_e = 324
    d['dx_strk'] = diag_rating_to_desc(int(row[strk_s:strk_e]))
    
    # Add Region
    # REGION15 = 79-80
    region_s = 78
    region_e = 80
    reg_rate = int(row[region_s:region_e])
    if reg_rate == -1:
        d['region'] = 'Inapplicable'
    elif reg_rate == 1:
        d['region'] = 'Northeast'
    elif reg_rate == 2:
        d['region'] = "Midwest"
    elif reg_rate == 3:
        d['region'] = "South"
    elif reg_rate == 4:
        d['region'] = 'West'
        
    # Add ER Expenditures
    # ERFEXP15 = 4222-4227
    ertot_s = 4221
    ertot_e = 4227
    d['ER_exp'] = int(row[ertot_s:ertot_e])
    
    # Add Outpatient Expenditures
    # OPFEXP15 = 3346-3351
    outp_exp_s = 3345
    outp_exp_e = 3351
    d['outp_exp'] = int(row[outp_exp_s:outp_exp_e])
    
    # Add Inpatient Expenditures (including 0-night visits)
    # IPFEXP15 = 4553-4558
    inp_exp_s = 4552
    inp_exp_e = 4558
    d['inp_exp'] = int(row[inp_exp_s:inp_exp_e])
    
    # Add Income level
    # POVLEV15 = 1433-1443
    inc_s = 1432
    inc_e = 1443
    povlev = float(row[inc_s:inc_e])

    d['inc_contin'] = povlev
    
    if povlev < 100:
        d['inc_categ'] = 'Poor'
    elif povlev < 125:
        d['inc_categ'] = 'Near Poor'
    elif povlev < 200:
        d['inc_categ'] = 'Low Income'
    elif povlev < 400:
        d['inc_categ'] = 'Middle Income'
    else:
        d['inc_categ'] = 'High Income'
    
    # Add Total Income
    # FAMINC15 = 1425-1431
    faminc_s = 1424
    faminc_e = 1431
    d['fam_inc_tot'] = int(row[faminc_s:faminc_e]) 
        
    return d

### Creating a DataFrame from Data

When using Pandas, the first step is creating a "DataFrame", which can be read from common tabular file formats like csv or created from python objects in different ways.  One way is to create a list of dictionaries, where each dictionary forms a row of the dataset, and each key-value pair fills a the value for that rows column, and pass that to pd.DataFrame().  A simple example is below.

In [12]:
restaurant_1 = {'name': 'ABCD',
                'style': 'French',
                'price': '$$$$$',
                'distance': 5}

restaurant_2 = {'name': 'EFGH',
                'style': 'American',
                'price': '$$',
                'distance': 8}

rest_df = pd.DataFrame([restaurant_1, restaurant_2])
rest_df.head()

,distance,name,price,style
0,5,ABCD,$$$$$,French
1,8,EFGH,$$,American


Construct a dataframe of the user_id, age & sex data in the MEPS, creating a list of dictionaries by applying your function from above.  Once you have accomplished this, add additional features to your parsing function that may be relevant to this project.  

We will at least want to know total spending (TOTEXP) and perceived health and mental status but also explore the codebook a bit and identify any other features you think may be relevant to health spending.

In [13]:
# Create your DataFrame here
with open("/Users/amy/Downloads/h181.dat") as data2015:
    lines = data2015.readlines()
    
meps_data15 = []

for line in lines:
    parsed_line = parse_row_2015(line)
    meps_data15.append(parsed_line)
    
meps_df15 = pd.DataFrame(meps_data15)
meps_df15.head()


,ER_exp,age,dx_COPD,dx_IHD,dx_arth,dx_asth,dx_canc,dx_diab,dx_hyplp,dx_hyptn,...,rh42_53_chnge,rh53,rh_tot_chnge,sex,tot_exp,tot_mcd_exp,tot_mcr_exp,tot_prv_exp,tot_slf_exp,user_id
0,772,53,Yes,Yes,Yes,No,No,Yes,Yes,Yes,...,0,Fair,0,male,46612,5917,40320,0,375,60001101
1,362,56,No,No,Yes,Yes,No,Yes,Yes,Yes,...,0,Fair,0,female,9207,6231,2959,0,11,60001102
2,0,30,No,No,No,No,No,No,No,No,...,0,Excellent,0,male,0,0,0,0,0,60001103
3,381,23,No,No,No,No,No,No,No,No,...,0,Excellent,0,female,808,786,0,0,22,60001104
4,0,3,Inapplicable,Inapplicable,Inapplicable,No,Inapplicable,Inapplicable,Inapplicable,Inapplicable,...,0,Excellent,0,male,2721,2379,0,0,77,60001105


##### Saving Parsed Data

Once you have constructed your parsed dataset, you can easily save it with the `to_csv()` DataFrame method. NOTE: in addition to the save path, pass the argument `index=False`, because this DataFrame does not have a meaningful index.

In [14]:
# Save your DF here
meps_df15.to_csv('mepsdata2015.csv', index=False)


## Generating 2014 Dataset

Using what you've learned above, download, parse, and save the MEPS data for 2014.  NOTE: field positions change, so you will need a new parsing function

In [10]:
def parse_row_2014(row):
    """
    Write a function to extract and label data from raw 2015 MEPS
    data
    """
    d = {}
    # Add age
    # AGELAST = 179-180
    age_s = 178
    age_e = 180
    d['age'] = int(row[age_s:age_e])
    
    # Add sex
    # SEX = 187
    sexpos = 186
    sexcode = int(row[sexpos])
    if sexcode == 1:
        d['sex'] = 'male'
    elif sexcode == 2:
        d['sex'] = 'female'
    
    # Add ID
    # DUPERSID = 9-16
    id_s = 8
    id_e = 16
    d['user_id'] = int(row[id_s:id_e])
    
    # Add Race
    # RACEV1X = 188
    racepos = 187
    racecode = int(row[racepos])
    if racecode == 1:
        d['race'] = 'White'
    elif racecode == 2:
        d['race'] = 'Black'
    elif racecode == 3:
        d['race'] = 'American Indian/Alaska Native'
    elif racecode == 4:
        d['race'] = 'Asian/Native Hawaiian/Pacific Islander'
    elif racecode == 6:
        d['race'] = 'Multiple Races Reported'
    
    # Add Total Expenditures
    # TOTEXP14 = 2584-2589
    totexp_s = 2583
    totexp_e = 2589
    d['total_exp'] = int(row[totexp_s:totexp_e])
    
    # Add Total out-of-pocket payments
    # TOTSLF14 = 2590-2594
    totslf_s = 2589
    totslf_e = 2594
    d['tot_slf_exp'] = int(row[totslf_s:totslf_e])
    
    # Add Total paid by Medicaid
    # TOTMCD14 = 2601-2606
    totmcd_s = 2600
    totmcd_e = 2606
    d['tot_mcd_exp'] = int(row[totmcd_s:totmcd_e])
    
    # Add Total paid by Medicare
    # TOTMCR14 = 2595-2600
    totmcr_s = 2594
    totmcr_e = 2600
    d['tot_mcr_exp'] = int(row[totmcr_s:totmcr_e])
    
    # Add Total paid by Private insurance
    # TOTPRV14 = 2607-2612
    totprv_s = 2606
    totprv_e = 2612
    d['tot_prv_exp'] = int(row[totprv_s:totprv_e])
    
    # Insurance coverage status
    # Insurance code to status helper function
    def insur_code_to_desc(code):
        if code == 1:
            return 'Yes'
        elif code == 2:
            return 'No'
        
    # Private Insurance (did you ever have private insurance during 2015?)
    # PRVEV14 = 2189
    # Is this the right metric to use?? Look into making this a categorical variable
    prvev = 2188
    prvev_code = int(row[prvev])
    d['insur_priv'] = insur_code_to_desc(prvev_code)
    
    # Medicare in 2014
    # MCREV14 = 2191
    mcrev = 2190
    mcrev_code = int(row[mcrev])
    d['insur_mcr'] = insur_code_to_desc(mcrev_code)
    
    # Medicaid in 2014
    # MCDEV14 = 2192
    mcdev = 2191
    mcdev_code = int(row[mcdev])
    d['insur_mcd'] = insur_code_to_desc(mcdev_code)
    
    
    # Perceived Health Rating helper function
    def self_rate_to_desc(rating):
        if rating == -9:
            return 'Not Ascertained'
        elif rating == -8:
            return 'DK'
        elif rating == -7:
            return 'Refused'
        elif rating == -1:
            return 'Inapplicable'
        elif rating == 1:
            return 'Excellent'
        elif rating == 2:
            return 'Very Good'
        elif rating == 3:
            return 'Good'
        elif rating == 4:
            return 'Fair'
        elif rating == 5:
            return 'Poor'
    
    # Add Perceived Health
    # RTHLTH31 = 287-288
    rh31_s = 286
    rh31_e = 288
    rh31_rate = int(row[rh31_s:rh31_e])
    d['rh31'] = self_rate_to_desc(rh31_rate)
    # RTHLTH42 = 289-290
    rh42_s = 288
    rh42_e = 290
    rh42_rate = int(row[rh42_s:rh42_e])
    d['rh42'] = self_rate_to_desc(rh42_rate)
    # RTHLTH53 = 291-292
    rh53_s = 290
    rh53_e = 292
    rh53_rate = int(row[rh53_s:rh53_e])
    d['rh53'] = self_rate_to_desc(rh53_rate)
    
    # Add Perceived Mental Health
    # MNHLTH31 = 293-294
    mh31_s = 292
    mh31_e = 294
    mh31_rate = int(row[mh31_s:mh31_e])
    d['mh31'] = self_rate_to_desc(mh31_rate)
    # MNHLTH42 = 295-296
    mh42_s = 294
    mh42_e = 296
    mh42_rate = int(row[mh42_s:mh42_e])
    d['mh42'] = self_rate_to_desc(mh42_rate)
    # MNHLTH53 = 297-298
    mh53_s = 296
    mh53_e = 298
    mh53_rate = int(row[mh53_s:mh53_e])
    d['mh53'] = self_rate_to_desc(mh53_rate)
    
    # Create rating columns to describe change in self-assessed health
    # Rating change helper function
    def sah_rating_change(rate1, rate2):
        if (rate1 < 1) | (rate2 < 1):
            return "N/A"
        else:
            return rate2 - rate1
    
    # RH31 to RH42 Change
    d['rh31_42_chnge'] = sah_rating_change(rh31_rate, rh42_rate)
    # RH42 to RH53 Change
    d['rh42_53_chnge'] = sah_rating_change(rh42_rate, rh53_rate)
    # Total RH change
    d['rh_tot_chnge'] = sah_rating_change(rh31_rate, rh53_rate)
    # MH31 to MH42 change
    d['mh31_42_chnge'] = sah_rating_change(mh31_rate, mh42_rate)
    # MH42 to MH53 change
    d['mh42_53_chnge'] = sah_rating_change(mh42_rate, mh53_rate)
    # Total MH change
    d['mh_tot_chnge'] = sah_rating_change(mh31_rate, mh53_rate)
    
    
    # Diagnosis Rating helper function
    def diag_rating_to_desc(rating):
        if rating == -9:
            return "Not Ascertained"
        elif rating == -8:
            return "DK"
        elif rating == -7:
            return "Refused"
        elif rating == -1:
            return "Inapplicable"
        elif rating == 1:
            return "Yes"
        elif rating == 2:
            return "No"
        
    # Add Arthritis Diagnosis
    # ARTHDX = 371-372
    arth_s = 370
    arth_e = 372
    d['dx_arth'] = diag_rating_to_desc(int(row[arth_s:arth_e]))
    
    # Add Asthma Diagnosis
    # ASTHDX = 377-378
    asth_s = 376
    asth_e = 378
    d['asthma'] = diag_rating_to_desc(int(row[asth_s:asth_e]))
    
    # Add Cancer Diagnosis
    # CANCERDX = 337-338
    canc_s = 336
    canc_e = 338
    d['cancer'] = diag_rating_to_desc(int(row[canc_s:canc_e]))
    
    # Add Copd (Chronic Obstructive Pulmonary Disease) Diagnosis (Emphysema)
    # EMPHDX = 325-326
    copd_s = 324
    copd_e = 326
    d['dx_COPD'] = diag_rating_to_desc(int(row[copd_s:copd_e]))
    
    # Add Hyperlipidemia Diagnosis (high cholesterol)
    # CHOLDX = 333-334
    hyplip_s = 332
    hyplip_e = 334
    d['dx_hyplp'] = diag_rating_to_desc(int(row[hyplip_s:hyplip_e]))
    
    # Add Diabetes Diagnosis
    # DIABDX = 363-364
    diab_s = 362
    diab_e = 364
    d['diabetes'] = diag_rating_to_desc(int(row[diab_s:diab_e]))

    # Add Hypertension Diagnosis (high blood pressure)
    # HIBPDX = 299-300
    hypt_s = 298
    hypt_e = 300
    d['dx_hyptn'] = diag_rating_to_desc(int(row[hypt_s:hypt_e]))

    # Add ischemic heart disease Diagnosis (coronary heart disease?)
    # CHDDX = 305-306
    ihd_s = 304
    ihd_e = 306
    d['dx_IHD'] = diag_rating_to_desc(int(row[ihd_s:ihd_e]))

    # Add stroke Diagnosis
    # STRKDX = 321-322
    strk_s = 320
    strk_e = 322
    d['dx_strk'] = diag_rating_to_desc(int(row[strk_s:strk_e]))
    
    
    
    # Add Region
    # REGION14 = 79-80
    region_s = 78
    region_e = 80
    reg_rate = int(row[region_s:region_e])
    if reg_rate == -1:
        d['region'] = 'Inapplicable'
    elif reg_rate == 1:
        d['region'] = 'Northeast'
    elif reg_rate == 2:
        d['region'] = "Midwest"
    elif reg_rate == 3:
        d['region'] = "South"
    elif reg_rate == 4:
        d['region'] = 'West'
    
    # Add Income level
    # POVLEV14 = 1454-1464
    inc_s = 1453
    inc_e = 1464
    povlev = float(row[inc_s:inc_e])

    d['inc_contin'] = povlev
    
    if povlev < 100:
        d['inc_categ'] = 'Poor'
    elif povlev < 125:
        d['inc_categ'] = 'Near poor'
    elif povlev < 200:
        d['inc_categ'] = 'Low Income'
    elif povlev < 400:
        d['inc_categ'] = 'Middle Income'
    else:
        d['inc_categ'] = 'High Income'
        
    # Add Total Income
    # FAMINC14 = 1447-1452
    faminc_s = 1446
    faminc_e = 1452
    d['fam_inc_tot'] = int(row[faminc_s:faminc_e]) 
        
    return d

# MEPS 2014 data parsing
with open("/Users/amy/Downloads/h171.dat") as data2014:
    lines14 = data2014.readlines()

meps_data14 = []
for line in lines14:
    parsed_line14 = parse_row_2014(line)
    meps_data14.append(parsed_line14)
    
meps_df14 = pd.DataFrame(meps_data14)
    
# Save your DF here
meps_df14.to_csv('mepsdata2014.csv', index=False)

meps_df14.head(15)


,age,asthma,cancer,diabetes,dx_COPD,dx_IHD,dx_arth,dx_hyplp,dx_hyptn,dx_strk,...,rh42_53_chnge,rh53,rh_tot_chnge,sex,tot_mcd_exp,tot_mcr_exp,tot_prv_exp,tot_slf_exp,total_exp,user_id
0,36,No,No,No,No,No,No,No,No,No,...,0,Very Good,0,male,0,0,1353,165,1518,40001101
1,36,No,No,No,No,No,No,No,No,No,...,-1,Excellent,-1,female,0,0,1082,152,1234,40001102
2,15,No,Inapplicable,Inapplicable,Inapplicable,Inapplicable,Inapplicable,Inapplicable,Inapplicable,Inapplicable,...,-1,Excellent,-1,female,0,0,536,135,671,40001103
3,8,No,Inapplicable,Inapplicable,Inapplicable,Inapplicable,Inapplicable,Inapplicable,Inapplicable,Inapplicable,...,-1,Excellent,-1,male,0,0,163,89,252,40001104
4,85,No,No,No,No,No,No,No,No,No,...,-2,Excellent,0,female,0,9279,1622,2,10904,40002101
5,34,No,No,No,No,No,No,No,No,No,...,0,Good,0,male,0,0,0,0,0,40004101
6,32,No,No,No,No,No,No,No,No,No,...,0,Good,0,female,0,0,0,12,43,40004102
7,15,No,Inapplicable,Inapplicable,Inapplicable,Inapplicable,Inapplicable,Inapplicable,Inapplicable,Inapplicable,...,0,Good,0,female,0,0,0,80,201,40004103
8,11,Yes,Inapplicable,Inapplicable,Inapplicable,Inapplicable,Inapplicable,Inapplicable,Inapplicable,Inapplicable,...,0,Good,0,male,3006,0,0,159,3189,40004104
9,9,No,Inapplicable,Inapplicable,Inapplicable,Inapplicable,Inapplicable,Inapplicable,Inapplicable,Inapplicable,...,-1,Very Good,-1,male,256,0,0,0,256,40004105


#### Next Notebook: Data Analysis with Pandas

Now that you have extracted MEPS and provided some structure to the data, we will introduce ways to explore and analyze the dataset with Pandas.